### Test methods that work with accounts

In [1]:
from dexter.DB import DB, Account, Entry, Transaction, Document, Tag

import re

In [2]:
DB.open('dev')

### Match Account Names

In [3]:
Account.objects(name__contains='checking')

[<Account: checking assets>]

In [4]:
Account.objects(name__contains='food')

[<Account: expenses:food expenses>, <Account: groceries expenses>, <Account: restaurant expenses>]

In [5]:
for a in DB.find_account('food'):
    print(a)

expenses:food expenses
groceries expenses
restaurant expenses


In [6]:
DB.account_name_parts()

{'assets',
 'bank',
 'car',
 'chase',
 'checking',
 'equity',
 'expenses',
 'food',
 'fuel',
 'groceries',
 'home',
 'household',
 'income',
 'liabilities',
 'mortgage',
 'payment',
 'restaurant',
 'savings',
 'travel',
 'visa',
 'yoyodyne'}

In [7]:
DB.account_name_parts('expenses')

{'car',
 'expenses',
 'food',
 'fuel',
 'groceries',
 'home',
 'household',
 'mortgage',
 'payment',
 'restaurant',
 'travel'}

In [9]:
dct = DB.account_names()

In [10]:
dct

{'equity': ['equity', 'equity', 'equity'],
 'assets': ['assets:bank:checking', 'assets:bank:savings'],
 'bank': ['assets:bank:checking', 'assets:bank:savings'],
 'checking': ['assets:bank:checking'],
 'savings': ['assets:bank:savings'],
 'expenses': ['expenses:car',
  'expenses:car:payment',
  'expenses:car:fuel',
  'expenses:food',
  'expenses:food:groceries',
  'expenses:food:restaurant',
  'expenses:home',
  'expenses:home:household',
  'expenses:home:mortgage',
  'expenses:travel'],
 'car': ['expenses:car', 'expenses:car:payment', 'expenses:car:fuel'],
 'payment': ['expenses:car:payment'],
 'fuel': ['expenses:car:fuel'],
 'food': ['expenses:food',
  'expenses:food:groceries',
  'expenses:food:restaurant'],
 'groceries': ['expenses:food:groceries'],
 'restaurant': ['expenses:food:restaurant'],
 'home': ['expenses:home',
  'expenses:home:household',
  'expenses:home:mortgage'],
 'household': ['expenses:home:household'],
 'mortgage': ['expenses:home:mortgage'],
 'travel': ['expenses:t

In [11]:
dct['expenses']

['expenses:car',
 'expenses:car:payment',
 'expenses:car:fuel',
 'expenses:food',
 'expenses:food:groceries',
 'expenses:food:restaurant',
 'expenses:home',
 'expenses:home:household',
 'expenses:home:mortgage',
 'expenses:travel']

In [12]:
len(dct['expenses'])

10

In [13]:
dct = DB.account_names('expenses')

In [14]:
dct

{'expenses': ['expenses:car',
  'expenses:car:payment',
  'expenses:car:fuel',
  'expenses:food',
  'expenses:food:groceries',
  'expenses:food:restaurant',
  'expenses:home',
  'expenses:home:household',
  'expenses:home:mortgage',
  'expenses:travel'],
 'car': ['expenses:car', 'expenses:car:payment', 'expenses:car:fuel'],
 'payment': ['expenses:car:payment'],
 'fuel': ['expenses:car:fuel'],
 'food': ['expenses:food',
  'expenses:food:groceries',
  'expenses:food:restaurant'],
 'groceries': ['expenses:food:groceries'],
 'restaurant': ['expenses:food:restaurant'],
 'home': ['expenses:home',
  'expenses:home:household',
  'expenses:home:mortgage'],
 'household': ['expenses:home:household'],
 'mortgage': ['expenses:home:mortgage'],
 'travel': ['expenses:travel']}

In [15]:
len(dct)

11

In [16]:
dct['car']

['expenses:car', 'expenses:car:payment', 'expenses:car:fuel']

In [17]:
DB.account_groups()

['equity',
 'assets:bank:checking',
 'assets:bank:savings',
 'expenses:car',
 'expenses:car:payment',
 'expenses:car:fuel',
 'expenses:food',
 'expenses:food:groceries',
 'expenses:food:restaurant',
 'expenses:home',
 'expenses:home:household',
 'expenses:home:mortgage',
 'expenses:travel',
 'income:yoyodyne',
 'liabilities:chase:visa',
 'equity',
 'equity']

In [18]:
DB.account_groups(['expenses'])

['expenses:car',
 'expenses:car:payment',
 'expenses:car:fuel',
 'expenses:food',
 'expenses:food:groceries',
 'expenses:food:restaurant',
 'expenses:home',
 'expenses:home:household',
 'expenses:home:mortgage',
 'expenses:travel']

In [19]:
DB.account_groups(['expenses:1'])

['expenses.*']

In [20]:
DB.account_groups(['expenses:2'])

['expenses:car.*', 'expenses:food.*', 'expenses:home.*', 'expenses:travel.*']

In [21]:
DB.account_groups(['expenses:3'])

['expenses:car',
 'expenses:car:payment.*',
 'expenses:car:fuel.*',
 'expenses:food',
 'expenses:food:groceries.*',
 'expenses:food:restaurant.*',
 'expenses:home',
 'expenses:home:household.*',
 'expenses:home:mortgage.*',
 'expenses:travel']

In [22]:
DB.account_groups(['expenses:food'])

['expenses:food', 'expenses:food:groceries', 'expenses:food:restaurant']

In [23]:
DB.account_groups(['expenses:food:2'])

['expenses:food.*']

In [24]:
DB.account_groups(['expenses:food:3'])

['expenses:food', 'expenses:food:groceries.*', 'expenses:food:restaurant.*']

In [25]:
DB.account_groups(['expenses','income'])

['expenses:car',
 'expenses:car:payment',
 'expenses:car:fuel',
 'expenses:food',
 'expenses:food:groceries',
 'expenses:food:restaurant',
 'expenses:home',
 'expenses:home:household',
 'expenses:home:mortgage',
 'expenses:travel',
 'income:yoyodyne']

In [26]:
DB.account_groups(['expenses:1','income:1'])

['expenses.*', 'income.*']

In [27]:
DB.account_groups(['expenses:2','income:2'])

['expenses:car.*',
 'expenses:food.*',
 'expenses:home.*',
 'expenses:travel.*',
 'income:yoyodyne.*']

In [4]:
Account.objects(name__exact='expenses:food:groceries')

[<Account: groceries expenses>]

In [5]:
Account.objects(abbrev__exact='groceries')

[<Account: groceries expenses>]

In [4]:
s = 'expenses:food:groceries'

In [7]:
if acct := Account.objects(name__exact=s) or Account.objects(abbrev__exact=s):
    print(acct[0].name)

expenses:food:groceries


In [3]:
DB.fullname('expenses:food:groceries')

'expenses:food:groceries'

In [4]:
DB.fullname('groceries')

'expenses:food:groceries'

In [5]:
DB.fullname('groc')

### Balances

In [27]:
DB.select(Entry, account='expenses:food*')

[<Entry: <En 2024-01-02 expenses:food -$500.0 [<Tag.B: '#budget'>]>>, <Entry: <En 2024-02-02 expenses:food -$500.0 [<Tag.B: '#budget'>]>>, <Entry: <En 2024-01-07 expenses:food:groceries +$75.0 []>>, <Entry: <En 2024-01-21 expenses:food:groceries +$175.0 []>>, <Entry: <En 2024-02-07 expenses:food:groceries +$75.0 []>>, <Entry: <En 2024-02-21 expenses:food:groceries +$75.0 []>>]

In [28]:
for e in DB.select(Entry, account='expenses:food*'):
    print(e)

<En 2024-01-02 expenses:food -$500.0 [<Tag.B: '#budget'>]>
<En 2024-02-02 expenses:food -$500.0 [<Tag.B: '#budget'>]>
<En 2024-01-07 expenses:food:groceries +$75.0 []>
<En 2024-01-21 expenses:food:groceries +$175.0 []>
<En 2024-02-07 expenses:food:groceries +$75.0 []>
<En 2024-02-21 expenses:food:groceries +$75.0 []>


In [29]:
for t in DB.select(Transaction, tag='#budget'):
    print(t)

<Tr 2024-01-02 expenses:food/expenses:home/expenses:car/expenses:travel/equity -> income:yoyodyne $5000.0 fill envelopes ['#budget']>
<Tr 2024-02-02 expenses:food/expenses:home/expenses:car/expenses:travel/equity -> income:yoyodyne $5000.0 fill envelopes ['#budget']>


In [35]:
DB.balance('food')

-600.0

In [36]:
DB.balance('food', budgets=False)

400.0

In [37]:
DB.balance('food', ending='2024-01-31')

-250.0

In [38]:
DB.balance('food', ending='2024-01-31', budgets=False)

250.0